
# M03 Homework

```yaml
Course:  DS 5001
Module:  M03 Language Models
Author:  JiHo Lee (qxz6hb)
```

### Question 1. List six words that precede the word "monster," excluding stop words (and sentence boundary markers). Stop words include 'a', 'an', 'the', 'this', 'that', etc. Hint: use the df.query() method.

> miserable, abhorred, detestable, hideous, hellish, gigantic

### Question 2. List the following sentences in ascending order of bigram perplexity according to the language model generated from the text:

> I have never seen the aurora borealis. <br> The monster is on the ice.  <br>
> He never knew the love of a family. <br>
> Flowers are happy things

### Question 3. Using the bigram model represented as a matrix, explore the relationship between bigram pairs using the following lists. H

> he   said     5.062035<br>he   heard    7.130229  
<br>she  said     6.771566
<br>she  heard    6.771566     

<b>''he said''</b> is the most likely bigram to occur in the corpus. <b>''he heard''</b> is the least likely bigram in the corpus. <b>''she said'', ''she heard''</b> have equal likelihoods.


### Question 4. Generate 20 sentences using the generate_text() function. Display the results.

> ```The generated sentences are described below in the according section with the code```


### Question 5. Compute the redundancy  for each of the n-gram models using the MLE of the joint probability of each ngram type. 

> Each redundancy for unigram, bigram, trigram is <b>0.309, 0.112, 0.0684</b> respectively.
<br> The redundancy _R_ decreases as the length of the n-gram increases.

```The code related to questions and answers with explanation is described in each
corresponding section below.```

### Set Up Environment

In [2]:
import pandas as pd
import numpy as np

import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_dir = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

data_dir  = data_dir.replace('/', '\\')
output_dir  = output_dir.replace('/', '\\')

ngram_order = 3
widx = [f"w{i}" for i in range(ngram_order)]

In [3]:
# data_dir = output_dir
OHCO = ['book_id','chap_id','para_num','sent_num','token_num']

text_file = f"{data_dir}/gutenberg/pg42324.txt"
csv_token  = f"{output_dir}/austen-combo-TOKENS.csv" # The file we will create
csv_vocab = f"{output_dir}/austen-combo-VOCAB.csv" 


text_file = text_file.replace('/', '\\')
csv_token = csv_token.replace('/', '\\')
csv_vocab = csv_vocab.replace('/', '\\')

from textimporter import TextImporter
from textparser import TextParser

src_file = text_file
# ohco_pats = [('chap', r'^(?:INTRODUCTION|PREFACE|LETTER|CHAPTER)\.?\b', 'm')]
ohco_pats = [('chap', r'^(?:INTRODUCTION|PREFACE|LETTER|CHAPTER)\b', 'm')]
clip_pats = [r'START', r'END']

test= TextImporter(src_file=src_file, ohco_pats=ohco_pats, clip_pats=clip_pats)
test.import_source().parse_tokens()

TOKEN = test.TOKENS
test.extract_vocab()
VOCAB = test.VOCAB

VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['modified_term_str'] = VOCAB.index
VOCAB.loc[(VOCAB.n == 1) & (VOCAB.n_chars < 3), 'modified_term_str'] = "<UNK>"

TOKEN['modified_term_str'] = TOKEN.term_str.map(VOCAB.modified_term_str)
# TOKEN[TOKEN.modified_term_str == '<UNK>'].sample(5)

def token_to_padded(token, grouper=['sent_num'], term_str='term_str'):
    ohco = token.index.names # We preserve these since they get lost in the shuffle'
    padded = token.groupby(grouper)\
        .apply(lambda x: '<s> ' + ' '.join(x[term_str]) + ' </s>')\
        .apply(lambda x: pd.Series(x.split()))\
        .stack().to_frame('term_str')
    padded.index.names = ohco
    return padded
def padded_to_ngrams(padded, grouper=['sent_num'], n=2):

    ohco = padded.index.names # This gets lost in the grouping
    G = padded.groupby(grouper)
    ng = [None for i in range(ngram_order)]
    for i in range(ngram_order):
        if i == 0:
            ng[0] = G.apply(lambda x: x)
        else:
            ng[i] = G.apply(lambda x: pd.concat([x.shift(i) for i in range(i,-1,-1)], axis=1))
        ng[i].columns = widx[:i+1]    
        ng[i] = ng[i].reset_index(drop=True)
        ng[i].index = padded.index
        ng[i] = ng[i].fillna('</s>')
            
    return ng
PADDED = token_to_padded(TOKEN, grouper=OHCO[1:4], term_str='modified_term_str')
PADDED = PADDED.unstack().reset_index(drop=True).stack().dropna()
PADDED.index.names = ['sent_num', 'token_num']

VOCAB2 = PADDED.term_str.value_counts().to_frame('n')
NG = padded_to_ngrams(PADDED, n=ngram_order)

Importing  C:\DS5001\DS5001_2024_01_R\..\data\gutenberg\pg42324.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^(?:INTRODUCTION|PREFACE|LETTER|CHAPTER)\b
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by delimitter [.?!;:]+
Parsing OHCO level 3 token_num by delimitter [\s',-]+


### <mark>Question 1.</mark>
miserable, abhorred, detestable, hideous, hellish, gigantic

In [4]:
stop = ['this', 'the', 'a', '<s>']
tmp = NG[1].query("w1=='monster'")
tmp2 = tmp[~tmp['w0'].isin(['this', 'the', 'a', '<s>'])]
tmp2

,,w0,w1
sent_num,token_num,,
1111,26,miserable,monster
2233,2,abhorred,monster
3035,24,detestable,monster
3254,2,hideous,monster
4719,6,hellish,monster
4817,3,gigantic,monster


### Generate NGram Models

In [5]:
def ngrams_to_models(ngrams, k=.01):

    model = [None for i in range(ngram_order)]
    K = len(VOCAB2) * k
    
    for i in range(ngram_order):    
        if i == 0:
            model[i] = ngrams[i].value_counts().to_frame('n')
            model[i]['p'] = (model[i].n + k) / (model[i].n.sum() + K)
            model[i]['i'] = np.log2(1/model[i].p)
        else:
            model[i] = ngrams[i].value_counts().to_frame('n')
            model[i]['cp'] = (model[i].n + k) / (model[i-1].n + K)
            model[i]['i'] = np.log2(1/model[i].cp)
        model[i] = model[i].sort_index()
            
    return model
M = ngrams_to_models(NG)

### Test Models

In [6]:
def ngrams_to_matrix_models(ngrams, k=.01):
    M = [None for i in range(ngram_order)]
    for i in range(ngram_order):
        M[i] = NG[i].value_counts()
        if i > 0:
            M[i] = M[i].unstack(fill_value=0).T + k
        else:
            M[i] = M[i].to_frame('i')
        M[i] = M[i] / M[i].sum()
        M[i] = -np.log2(M[i])        
    return M

S= "The monster is on the ice.\nFlowers are happy things.\nI have never seen the aurora borealis.\nHe never knew the love of a family."
S = S.split('\n')
S = pd.DataFrame(S, columns=['sent_str'])
S.index.name='sent_num'

# Convert dataframe of sentences to TOKEN with normalized terms
K = S.sent_str.apply(lambda x: pd.Series(x.split())).stack().to_frame('token_str')
K['term_str'] = K.token_str.str.replace(r"[\W_]+", "", regex=True).str.lower()
K.index.names = ['sent_num', 'token_num']

TEST_SENTS, TEST_TOKENS = S, K

TEST_TOKENS.loc[~TEST_TOKENS.term_str.isin(VOCAB2.index), 'term_str'] = "<UNK>"

TEST_PADDED = token_to_padded(TEST_TOKENS)
TEST_NG = padded_to_ngrams(TEST_PADDED, 'sent_num', ngram_order)

def test_model(model, ngrams, sents):
        
    n = len(model)
    ohco = ngrams[0].index.names
    
    max_i = np.log2(len(M[0])) # For unseen combos
    
    R = []
    for i in range(n):

        cols = widx[:i+1]
            
        T = ngrams[i].merge(M[i], on=cols, how='left')
        T.index = ngrams[i].index
        T = T.reset_index().set_index(ohco + cols).i
        
        # This how we handle unseen combos
        T[T.isna()] = max_i

        R.append(T.to_frame(f"i{i}"))
                
    return R

R = test_model(M, TEST_NG, TEST_SENTS)

### Sentence Perplexity

### <mark>Question 2.</mark>

I have never seen the aurora borealis.

The monster is on the ice. 

He never knew the love of a family.	

Flowers are happy things

In [7]:
pp_cols = []
mean_i_cols = []
for i in range(ngram_order):
    pp_col = f"pp{i}"
    mean_i_col = f"mean_i_{i}"
    pp_cols.append(pp_col)
    mean_i_cols.append(mean_i_col)
    TEST_SENTS[mean_i_col] = R[i].groupby('sent_num')[f"i{i}"].mean()
    TEST_SENTS[f"pp{i}"] = np.exp2(TEST_SENTS[mean_i_col])

TEST_SENTS.sort_values("pp1").style.background_gradient(cmap="YlGnBu", axis=0)

,sent_str,mean_i_0,pp0,mean_i_1,pp1,mean_i_2,pp2
sent_num,,,,,,,
2,I have never seen the aurora borealis.,8.228776,299.991202,5.525197,46.052165,9.306330,633.117678
0,The monster is on the ice.,6.870571,117.016718,5.593021,48.268880,8.573922,381.072693
3,He never knew the love of a family.,7.426112,171.981820,6.687920,103.101378,9.930102,975.570260
1,Flowers are happy things.,9.110441,552.733765,9.343909,649.825583,12.810973,7186.000000


### <mark>Question 3.</mark>

he   said     5.062035

he   heard    7.130229
     
she  said     6.771566

she  heard    6.771566     

<b>''he said''</b> is the most likely bigram to occur in the corpus. <b>''he heard''</b> is the least likely bigram in the corpus. <b>''she said'', ''she heard''</b> have equal likelihoods.

In [8]:
MM = ngrams_to_matrix_models(NG)

tmp = MM[1].unstack()
indices = ['he', 'she']
columns = ['said', 'heard']

relationships = tmp.loc[indices, columns]
relationships

w0   w1   
he   said     5.062035
     heard    7.130229
she  said     6.771566
     heard    6.771566
dtype: float64

### Generate Text

### <mark>Question 4.</mark>

The generated sentences are in the result below.

In [10]:
def generate_text(M, n=290):
    
    if len(M) < 3:
        raise ValueError("Must have trigram model generated.")
        
    words = ['</s>', '<s>']
    
    for i in range(n):
        
        bg = tuple(words[-2:])

        # Try trigram model
        try:
            next_word = M[2].loc[bg].sample(weights='cp').index[0]

        # If not found in model, back off to bigram model
        except KeyError as e1:
            # print(bg)
            try:
                # Get the last word in the bigram
                ug = bg[1]
                next_word = M[1].loc[ug].sample(weights='cp').index[0]
            
            # ... back off to unigram
            except KeyError as e2:
                # print(ug)
                next_word = M[0].sample(weights='p').index[0][0]
        
        words.append(next_word)
    
    text = ' '.join(words[2:])
    print('\n'.join([f"{i+1}. " +  line.replace('</s> <s>','').replace('<s>', '').strip().upper() 
                       for i, line in enumerate(text.split('</s>'))]))
generate_text(M)

1. I SHALL PERISH IN THE GREAT AND SUDDEN CHANGE
2. HE SAW THE BEAUTIES OF NATURE BUT THAT SHE MADE ANY PROFESSIONS
3. VICTOR HE IS NOT MADE OF NO HOPE OR CONSOLATION
4. AND I WILL GIVE YOU PAIN
5. MY FATHER HAD TAKEN PLACE
6. I DARED NOT
7. YET YOU WOULD NOT FOREGO THEIR HOLD
8. I CANNOT BEAR TO LIVE AND BE OVERWHELMED BY DISAPPOINTMENTS
9. I AM ONLY LEFT TO MY OLD HABITS
10. AND VERY FAIR
11. I THEN WAS IN THE DIFFERENT FEELINGS
12. WE ACCORDINGLY LAY TO HOPING THAT SOME ONE SHOULD GO DOWN TO POSTERITY
13. A PENETRATION INTO THE SEA OR RATHER THE VAST RIVER OF ICE IS NOT THUS INTERRUPTED THE BEING TO ME THE DUTY OF OBEYING THE DYING REQUEST OF MY HIDEOUS NARRATION
14. THE OLD MAN WAS RECOMMENCING HIS MUSIC WHEN SOME ONE TAPPED AT THE ESCAPE OF THEIR VIRTUES AND A DESIRE TO ATTAIN BREATHE A DEGREE OF COMPOSURE
15. AND DEPOSITING THE REMAINS OF HIS COMPANION
16. HOW ALL THE FIRMNESS OF WHICH WAS TO BE VISITED BY PEACEFUL DREAMS
17. IT WAS YOUR JOURNAL OF WHAT HAD HAPPENED
18. THE NEXT 

### <mark>Question 5.</mark>

Each redundancy for unigram, bigram, trigram is <b>0.309, 0.112, 0.0684</b> respectively.

The redundancy _R_ decreases as the length of the n-gram increases.

In [11]:
H_unigram = -np.sum(M[0]['p'] * np.log2(M[0]['p']))

num_unique_unigrams = len(M[0])
H_max_unigram = np.log2(num_unique_unigrams)

R_unigram = 1 - (H_unigram / H_max_unigram)

M_Orig = M
R_unigram 

0.3091704418633189

In [12]:
total_tokens = M[1]['n'].sum()
M[1]['prob'] = M[1]['n'] / total_tokens
M[1]['joint_prob'] = M[1]['prob']

H_bigram = -np.sum(M[1]['joint_prob'] * np.log2(M[1]['prob']))
num_unique_bigrams = len(M[1])
H_max_bigram = np.log2(num_unique_bigrams)

R_bigram = 1 - (H_bigram / H_max_bigram)

R_bigram

0.11177428801648903

In [13]:
total_tokens = M[2]['n'].sum()
M[2]['prob'] = M[2]['n'] / total_tokens
M[2]['joint_prob'] = M[2]['prob']

H_trigram = -np.sum(M[2]['joint_prob'] * np.log2(M[2]['prob']))
num_unique_trigrams = len(M[2])
H_max_trigram = np.log2(num_unique_trigrams)

R_trigram = 1 - (H_trigram / H_max_trigram)

R_trigram 

0.06838180312642583